In [53]:
import os

def parse_exp_out_content(filepath):
    """Parses the content of exp.out, returns a tuple (注释, 代码)."""
    with open(filepath, 'r') as file:
        lines = file.readlines()
        
        # Separating lines based on whether they start with an asterisk
        comments = [line for line in lines if line.startswith("#")]
        code = [line for line in lines if not line.startswith("#")]
        
    return ("".join(comments), "".join(code))

def process_folder(folder_path):
    """Process the given folder, and returns the required dict."""
    data = {}
    
    # Check if the given path is a directory
    if os.path.isdir(folder_path):
        # Iterate over all subdirectories
        for subfolder in os.listdir(folder_path):
            subfolder_path = os.path.join(folder_path, subfolder)
            
            # Check if it's a directory and if 'exp.out' exists in it
            if os.path.isdir(subfolder_path) and "exp.out" in os.listdir(subfolder_path):
                exp_out_path = os.path.join(subfolder_path, "exp.out")
                
                # Check if the file is empty
                if os.path.getsize(exp_out_path) == 0:
                    data[subfolder] = None
                else:
                    data[subfolder] = parse_exp_out_content(exp_out_path)
    return data




In [54]:
def extract_n_m_from_key(key):
    """Extracts n and m from the given key in the format en-pm-hx."""
    parts = key.split('-')
    n = int(parts[0][1])
    m = int(parts[1][1])
    x = int(parts[2][1])
    return n, m, x

def generate_python_functions(data, out_file="functions.py"):
    """Generates the required python functions based on the given data."""
    ires_to_content = {}

    for key, tmp in data.items():
        comments, code = tmp
        if len(code.strip()) == 0:
            continue

        n, m, x = extract_n_m_from_key(key)
        ires = 2 * m - 1 + n

        is_first_line = False

        if ires not in ires_to_content:
            ires_to_content[ires] = {}
            is_first_line = True

        if x not in ires_to_content[ires]:
            ires_to_content[ires][x] = []

        for comment in comments.split("\n"):
            ires_to_content[ires][x].append(comment)

        code_lines = code.split("\n")
        for i, line in enumerate(code_lines):
            if is_first_line:
                # Add the comments to the first line
                ires_to_content[ires][x].append(line)
                is_first_line = False
            else:
                ires_to_content[ires][x].append((line))
        ires_to_content[ires][x].append("\n")

    output = []

    for ires in sorted(ires_to_content.keys()):
        if ires >= 0:
            func_name = f"def res_{ires}(cc_obj, amp):"
        else:
            func_name = "def ene(cc_obj, amp):"
        
        output.append(func_name)
        is_first_line = True
        is_first_comment_line = False
        for x in sorted(ires_to_content[ires].keys()):
            for line in ires_to_content[ires][x]:
                if line.startswith("#"):
                    if is_first_comment_line:
                        output.append("\n    " + line)
                        is_first_comment_line = False
                    else:
                        output.append("    " + line)
                else:
                    if line.strip() == "":
                        continue

                    if is_first_line:
                        output.append("    res  = " + line)
                        is_first_line = False
                    else:
                        output.append("    res += " + line)
                    is_first_comment_line = True

        output.append("    return res\n\n")

    out = "\n".join(output)

    with open(out_file, "w") as file:
        file.write("import functools, numpy\n")
        file.write("einsum = functools.partial(numpy.einsum, optimize=True)\n\n")

        file.write(out)

In [55]:
# Test the function
folder_name = "epcc-e1-p2-h3"  # Example folder name
result = process_folder(folder_name)
generate_python_functions(result, out_file="_".join(folder_name.split("-")) + ".py")

-1.000000 * einsum('ji,IJaj->IJai',   cc_obj.h1e.oo, amp[4])
 1.000000 * einsum('ab,IJbi->IJai',   cc_obj.h1e.vv, amp[4])
 1.000000 * einsum('IK,JKai->IJai',      cc_obj.hpp, amp[4])
 1.000000 * einsum('JK,IKai->IJai',      cc_obj.hpp, amp[4])
-1.000000 * einsum('Iji,Jaj->IJai', cc_obj.h1p1e.oo, amp[2])
 1.000000 * einsum('Iab,Jbi->IJai', cc_obj.h1p1e.vv, amp[2])
-1.000000 * einsum('Jji,Iaj->IJai', cc_obj.h1p1e.oo, amp[2])
 1.000000 * einsum('Jab,Ibi->IJai', cc_obj.h1p1e.vv, amp[2])


In [ ]:
# Test the function
folder_name = "epcc-e1-p4-h3"  # Example folder name
result = process_folder(folder_name)
generate_python_functions(result, out_file="_".join(folder_name.split("-")) + ".py")